In [1]:
from robot_descriptions.a1_description import URDF_PATH
from darli.backend import PinocchioBackend, CasadiBackend, JointType
from darli.modeling import Robot, Functional
from darli.modeling.integrators import ForwardEuler, RK4, MidPoint
import numpy as np

In [2]:
dt = 1e-3
n_steps = 20
control_sampling = 5e-3
samples = int(dt * n_steps / control_sampling)

In [3]:
model = Functional(CasadiBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [4]:
np.random.seed(0)
tau = np.random.randn(model.nu)

tau

array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
       -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
        0.14404357,  1.45427351])

In [5]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

x0.shape, model.nq, model.nv

((37,), 19, 18)

## Functional way

In [6]:
model.state_space.integrator

In [7]:
euler_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [8]:
%timeit euler_roll()

551 µs ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
model.state_space.set_integrator(MidPoint)

midpoint_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [10]:
%timeit midpoint_roll()

1.13 ms ± 34.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
model.state_space.set_integrator(RK4)

rk4_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [12]:
%timeit rk4_roll()

2.22 ms ± 45 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pinocchio way

In [13]:
model = Robot(PinocchioBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [14]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

np.random.seed(0)
tau = np.random.randn(model.nu, samples)

In [15]:
model.state_space.integrator

In [16]:
model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

array([ 3.04277252e-04, -2.01330463e-04, -1.90248413e-03, -1.17040266e-02,
       -5.15787389e-04, -3.94341397e-04,  9.99931295e-01,  3.22966013e-02,
        1.50246560e-02, -1.98648484e-02,  2.99230958e-02,  5.13920733e-02,
       -1.17458362e-01,  2.93587807e-02,  5.98572029e-02, -1.30787434e-01,
        2.92957256e-02,  2.78096767e-04, -1.22598234e-02,  1.79929644e-02,
       -1.11581574e-02, -2.01115830e-01, -1.86671007e+00, -7.24765145e-02,
       -8.40182063e-02,  2.81446975e+00, -2.83148081e-01,  1.99502364e+00,
        2.41199628e+00,  2.77334598e+00, -6.48657322e+00,  2.17138080e+00,
        4.52850880e+00, -9.88360657e+00,  2.22627337e+00,  3.21107178e-01,
       -1.38804941e+00])

In [17]:
%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

AssertionError: We expect controls to have shape[1] = 1, but got 4

In [ ]:
euler_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_state, "quaternion norm", np.linalg.norm(euler_state[3:7])

(array([ 3.76576015e-04, -5.90776756e-04, -1.86484692e-02, -3.55866284e-02,
         2.98828478e-04, -2.06836938e-03,  9.99364410e-01,  1.20123327e-01,
         8.94504016e-02, -2.04843323e-01,  5.97518327e-02, -2.64078009e-02,
        -1.08357083e-02,  9.61331223e-02,  9.85161233e-02, -1.30548863e-01,
         8.77269278e-02, -3.11990450e-02,  7.02278288e-02,  1.64054750e-02,
         2.85389016e-02, -6.01619857e-01, -1.77246306e+00, -3.42885311e-02,
        -1.46697075e-01,  2.98211623e+00,  3.88621024e+00, -8.52350431e+00,
         1.48212236e+00, -2.21680733e+00,  4.25183734e+00,  2.29552203e+00,
         3.49743072e+00, -5.23065688e+00,  2.12015156e+00,  1.95744729e-01,
        -6.94286048e-01]),
 'quaternion norm',
 1.0)

In [ ]:
model.state_space.set_integrator(MidPoint)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

In [ ]:
midpoint_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_state, "quaternion norm", np.linalg.norm(midpoint_state[3:7])

In [ ]:
model.state_space.set_integrator(RK4)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

3.7 ms ± 40.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
rk4_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_state, "quaternion norm", np.linalg.norm(rk4_state[3:7])

AssertionError: We expect controls to have shape[1] = 1, but got 15